Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.png)

## Authentication in Azure Machine Learning

This notebook shows you how to authenticate to your Azure ML Workspace using

 1. Interactive Login Authentication
 2. Azure CLI Authentication
 3. Managed Service Identity (MSI) Authentication
 4. Service Principal Authentication
 5. Token Authentication
 
The interactive authentication is suitable for local experimentation on your own computer. Azure CLI authentication is suitable if you are already using Azure CLI for managing Azure resources, and want to sign in only once. The MSI and Service Principal authentication are suitable for automated workflows, for example as part of Azure Devops build.

In [ ]:
from azureml.core import Workspace

### Interactive Authentication

Interactive authentication is the default mode when using Azure ML SDK.

When you connect to your workspace using workspace.from_config, you will get an interactive login dialog.

In [ ]:
ws = Workspace.from_config()

Also, if you explicitly specify the subscription ID, resource group and workspace name, you will get the dialog.

In [ ]:
ws = Workspace(subscription_id="my-subscription-id",
               resource_group="my-ml-rg",
               workspace_name="my-ml-workspace")

Note the user you're authenticated as must have access to the subscription and resource group. If you receive an error

```
AuthenticationException: You don't have access to xxxxxx-xxxx-xxx-xxx-xxxxxxxxxx subscription. All the subscriptions that you have access to = ...
```

check that the you used correct login and entered the correct subscription ID.

In some cases, you may see a version of the error message containing text: ```All the subscriptions that you have access to = []```

In such a case, you may have to specify the tenant ID of the Azure Active Directory you're using. An example would be accessing a subscription as a guest to a tenant that is not your default. You specify the tenant by explicitly instantiating _InteractiveLoginAuthentication_ with Tenant ID as argument. The Tenant ID can be found, for example, from https://portal.azure.com under **Azure Active Directory**,  **Properties** as Directory ID.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="my-tenant-id")

ws = Workspace(subscription_id="my-subscription-id",
               resource_group="my-ml-rg",
               workspace_name="my-ml-workspace",
               auth=interactive_auth)

Despite having access to the workspace, you may sometimes see the following error when retrieving it:

```
You are currently logged-in to xxxxxxxx-xxx-xxxx-xxxx-xxxxxxxxxxxx tenant. You don't have access to xxxxxx-xxxx-xxx-xxx-xxxxxxxxxx subscription, please check if it is in this tenant.
```

This error sometimes occurs when you are trying to access a subscription to which you were recently added. In this case, you need to force authentication again to avoid using a cached authentication token that has not picked up the new permissions. You can do so by setting `force=true` on the `InteractiveLoginAuthentication()` object's constructor as follows:

In [ ]:
forced_interactive_auth = InteractiveLoginAuthentication(tenant_id="my-tenant-id", force=True)

ws = Workspace(subscription_id="my-subscription-id",
               resource_group="my-ml-rg",
               workspace_name="my-ml-workspace",
               auth=forced_interactive_auth)

### Azure CLI Authentication

If you have installed azure-cli package, and used ```az login``` command to log in to your Azure Subscription, you can use _AzureCliAuthentication_ class.

Note that interactive authentication described above won't use existing Azure CLI auth tokens. 

In [ ]:
from azureml.core.authentication import AzureCliAuthentication

cli_auth = AzureCliAuthentication()

ws = Workspace(subscription_id="my-subscription-id",
               resource_group="my-ml-rg",
               workspace_name="my-ml-workspace",
               auth=cli_auth)

print("Found workspace {} at location {}".format(ws.name, ws.location))

### MSI Authentication

__Note__: _MSI authentication is supported only when using SDK from Azure Virtual Machine. The code below will fail on local computer._

When using Azure ML SDK on Azure Virtual Machine (VM), you can use Managed Service Identity (MSI) based authentication. This mode allows the VM connect to the Workspace without storing credentials in the Python code.

As a prerequisite, enable System-assigned Managed Identity for your VM as described in [Configure managed identities for Azure resources on a VM using the Azure portal](https://docs.microsoft.com/azure/active-directory/managed-identities-azure-resources/qs-configure-portal-windows-vm).

Then, assign the VM access to your Workspace. For example from Azure Portal, navigate to your workspace, select __Access Control (IAM)__, __Add Role Assignment__,  specify __Virtual Machine__ for __Assign Access To__ dropdown, and select your VM's identity.

![msi assignment](images/msiaccess.PNG)

After completing these steps, you can use authenticate using MsiAuthentication instance.

In [ ]:
from azureml.core.authentication import MsiAuthentication

msi_auth = MsiAuthentication()

ws = Workspace(subscription_id="my-subscription-id",
               resource_group="my-ml-rg",
               workspace_name="my-ml-workspace",
               auth=msi_auth)

print("Found workspace {} at location {}".format(ws.name, ws.location))

### Service Principal Authentication

When setting up a machine learning workflow as an automated process, we recommend using Service Principal Authentication. This approach decouples the authentication from any specific user login, and allows managed access control.

Note that you must have administrator privileges over the Azure subscription to complete these steps.

The first step is to create a service principal. First, go to [Azure Portal](https://portal.azure.com), select **Azure Active Directory** and **App Registrations**. Then select **+New application**, give your service principal a name, for example _my-svc-principal_. You can leave other parameters as is.

Then click **Register**.

![service principal creation](images/svc-pr-1.PNG)

From the page for your newly created service principal, copy the _Application ID_ and _Tenant ID_ as they are needed later.
![application and tenant id](images/svc-pr-2.PNG)

Then select **Certificates & secrets**, and **+New client secret** write a description for your key, and select duration. Then click **Add**, and copy the value of client secret to a secure location.


![tenant id](images/svc-pr-3.PNG)

Finally, you need to give the service principal permissions to access your workspace. Navigate to **Resource Groups**, to the resource group for your Machine Learning Workspace. 

Then select **Access Control (IAM)** and **Add a role assignment**. For _Role_, specify which level of access you need to grant, for example _Contributor_. Start entering your service principal name and once it is found, select it, and click **Save**.

![add role](images/svc-pr-4.PNG)

Now you are ready to use the service principal authentication. For example, to connect to your Workspace, see code below and enter your own values for tenant ID, application ID, subscription ID, resource group and workspace.

**We strongly recommended that you do not insert the secret password to code**. Instead, you can use environment variables to pass it to your code, for example through Azure Key Vault, or through secret build variables in Azure DevOps. For local testing, you can for example use following PowerShell command to set the environment variable.

```
$env:AZUREML_PASSWORD = "my-password"
```

In [ ]:
import os
from azureml.core.authentication import ServicePrincipalAuthentication

svc_pr_password = os.environ.get("AZUREML_PASSWORD")

svc_pr = ServicePrincipalAuthentication(
    tenant_id="my-tenant-id",
    service_principal_id="my-application-id",
    service_principal_password=svc_pr_password)


ws = Workspace(
    subscription_id="my-subscription-id",
    resource_group="my-ml-rg",
    workspace_name="my-ml-workspace",
    auth=svc_pr
    )

print("Found workspace {} at location {}".format(ws.name, ws.location))

See [Register an application with the Microsoft identity platform](https://docs.microsoft.com/en-us/azure/active-directory/develop/quickstart-register-app) quickstart for more details about application registrations. 

### Token Authentication

When token generation and its refresh needs to be outside on AML SDK, we recommend using Token Authentication. It can be used for getting token for AML or ARM audience. Thus giving more granular control over token generated.

This authentication class requires users to provide method `get_token_for_audience` which will be called to retrieve the token based on the audience passed.

Audience that is passed to `get_token_for_audience` can be ARM or AML. Exact value that will be passed as audience will depend on cloud and type for audience.

In [ ]:
from azureml.core.authentication import TokenAuthentication, Audience

# This is a sample method to retrieve token and will be passed to TokenAuthentication
def get_token_for_audience(audience):
    from adal import AuthenticationContext
    client_id = "my-client-id"
    client_secret = "my-client-secret"
    tenant_id = "my-tenant-id"
    auth_context = AuthenticationContext("https://login.microsoftonline.com/{}".format(tenant_id))
    resp = auth_context.acquire_token_with_client_credentials(audience,client_id,client_secret)
    token = resp["accessToken"]
    return token


token_auth = TokenAuthentication(get_token_for_audience=get_token_for_audience)

ws = Workspace(
    subscription_id="my-subscription-id",
    resource_group="my-ml-rg",
    workspace_name="my-ml-workspace",
    auth=token_auth
    )

print("Found workspace {} at location {}".format(ws.name, ws.location))

token_aml_audience = token_auth.get_token(Audience.aml)
token_arm_audience = token_auth.get_token(Audience.arm)

# Value of audience pass to `get_token_for_audience` can be retrieved as follows:
# aud_aml_val = token_auth.get_aml_resource_id() # For AML
# aud_arm_val = token_auth._cloud_type.endpoints.active_directory_resource_id # For ARM


Token authentication object can be used to retrieve token for either AML or ARM audience,
which can be used by other clients to authenticate to AML or ARM

## Using Secrets in Remote Runs

Sometimes, you may have to pass a secret to a remote run, for example username and password to authenticate against external data source.

Azure ML SDK enables this use case through Key Vault associated with your workspace. The workflow for adding a secret is following.

On local computer:

 1. Read in a local secret, for example from environment variable or user input. To keep them secret, do not insert secret values into code as hard-coded strings.
 2. Obtain a reference to the keyvault
 3. Add the secret name-value pair in the key vault.
 
The secret is then available for remote runs as shown further below.

__Note__: The _azureml.core.keyvault.Keyvault_ is different from _azure.keyvault_ library. It is intended as simplified wrapper for setting, getting and listing user secrets in Workspace Key Vault.

In [ ]:
import uuid

local_secret = os.environ.get("LOCAL_SECRET", default = str(uuid.uuid4())) # Use random UUID as a substitute for real secret.
keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="secret-name", value = local_secret)

The _set_secret_ method adds a new secret if one doesn't exist, or updates an existing one with new value.

You can list secret names you've added. This method doesn't return the values of the secrets.

In [ ]:
keyvault.list_secrets()

You can retrieve the value of the secret, and validate that it matches the original value. 

__Note__: This method returns the secret value. Take care not to write the the secret value to output.

In [ ]:
retrieved_secret = keyvault.get_secret(name="secret-name")
local_secret==retrieved_secret

In submitted runs on local and remote compute, you can use the get_secret method of Run instance to get the secret value from Key Vault. 

The method gives you a simple shortcut: the Run instance is aware of its Workspace and Keyvault, so it can directly obtain the secret without you having to instantiate the Workspace and Keyvault within remote run.

__Note__: This method returns the secret value. Take care not to write the secret to output.

For example, let's create a simple script _get_secret.py_ that gets the secret we set earlier. In an actual appication, you would use the secret, for example to access a database or other password-protected resource.

In [ ]:
%%writefile get_secret.py

from azureml.core import Run

run = Run.get_context()
secret_value = run.get_secret(name="secret-name")
print("Got secret value {} , but don't write it out!".format(len(secret_value) * "*"))

Then, submit the script as a regular script run, and find the obfuscated secret value in run output. You can use the same approach to other kinds of runs, such as Estimator ones.

In [ ]:
from azureml.core import Experiment
from azureml.core.script_run_config import ScriptRunConfig

exp = Experiment(workspace = ws, name="try-secret")
src = ScriptRunConfig(source_directory=".", script="get_secret.py")

run = exp.submit(src)
run.wait_for_completion(show_output=True)

Furthermore, you can set and get multiple secrets using set_secrets and get_secrets methods.